# pre-requesties #

In [2]:
import time as tt
import numpy as np
import scipy as sp
import matplotlib.pyplot as pl
# from numba import njit, prange
# from numba import njit, prange


# Full Construction #

In [2]:

def fc(a,j,L):
    spin = np.array([[0,1],[1,0]]), np.array([[0,-1j],[+1j,0]]), np.array([[1,0],[0,-1]]) 
    
    out = 1
    
    for _ in range(j-1):
        #JW = np.kron(np.eye(2**(j-1)), np.kron(spin[abs(int(a)-1)], np.eye(2**(L-j))))
        out = np.kron(out,spin[2])
    
    op = 0.5*(spin[0]+a*1j*spin[1])
    # out = np.kron(out, np.kron(op, np.eye(2**(L-np.mod(j,L)-1))))
    out = np.kron(out, np.kron(op, np.eye(2**(L-j))))
    
    return out


def fermion_ham(V, L, J=1, BC=True):
    out = np.zeros((2**L,2**L), dtype=np.complex_)
    
    for n in range(1,L):
        
        out += -J*fc(+1,n,L) @ fc(-1,n+1,L) 
        out += -J*fc(+1,n+1,L) @ fc(-1,n,L)
        out += -0.5*V*( fc(+1,n,L) @ fc(-1,n,L) @ fc(+1,n+1,L) @ fc(-1,n+1,L))
        
    if BC:
        out += -J*fc(+1,L,L) @ fc(-1,1,L) 
        out += -J*fc(+1,1,L) @ fc(-1,L,L)
        out += -0.5*V*( fc(+1,L,L) @ fc(-1,L,L) @ fc(+1,1,L) @ fc(-1,1,L) )
    
    return out 


def full_total_particle_counts(L):
    
    out = np.zeros((2**L,2**L), dtype=np.complex_)  
    for n in range(1,L+1):    
        out += fc(+1,n,L) @ fc(-1,n,L)     
    
    return out


def full_particle_counts(input_state,L):
    
    input_state = np.array(input_state)
    # N = np.zeros((2**L,2**L), dtype=np.complex_) 
    Ns = np.zeros((L,) ) 
    for n in range(1,L+1):    
        n_th = fc(+1,n,L) @ fc(-1,n,L) 
        Ns[int(n-1)] += input_state @ n_th @ input_state.conj()
        
    return Ns


# 1D: Sector Construction #

In [131]:
def bit_flip(input, indx1, indx2):
    temp_state = np.array(input[:])
    temp_state[ [indx1,indx2] ] = 1 - temp_state[ [indx1,indx2] ]
    return temp_state

def acting_ham( n:int, V, J, L:int):
    state = []
    weight = []
    # state_weight = []
    
    bin_state = np.array([int(i) for i in np.binary_repr(n, width=L)])
    rolo_state = np.roll(bin_state, -1)
    # diag = - 0.5 * V * (4* rolo_state @ bin_state - 4*sum(bin_state) + L)
    diag = - 0.5 * V * (1* rolo_state @ bin_state )
    # state_weight.append((n, float(diag)))
    state.append(n)
    weight.append(float(diag))
    # condi = np.bitwise_xor(bin_state,np.roll(bin_state,-1))
    for l in range(L):
        if bin_state[ l ] != rolo_state[ l ]:
            new_state = bit_flip(bin_state, np.mod(l,L), np.mod(l+1,L))
            m = int("".join(str(i) for i in new_state),2)
            # sign_factor = int( abs(n-m)/( 2**(L-1)-1 ) )
            # state_weight.append(( m ,-1.0*J*(-1)**sign_factor ))
            sign_factor = -1 if sum(bin_state) % 2 ==0 and int(abs(n-m)/(2**(L-1)-1))==1 else +1.0
            # state_weight.append(( m , 1.0 * J * sign_factor ))
            state.append( m )
            weight.append( 1.0 * J * sign_factor )

    # return state_weight
    return state, weight


def basis_set_N(N,L):
    
    basis_N = []
    for n in range(2**L):
        bin_state = np.array([int(i) for i in np.binary_repr(n, width=L)])
        if sum(bin_state) == N:
            basis_N.append(n)
        
    return basis_N


def build_hamiltonian(N,L, parameters):
    
    V, J = parameters
    
    basis_N = basis_set_N(N,L)
    HN = np.zeros( (np.size(basis_N), np.size(basis_N)) )
    
    for ind1, n in enumerate(basis_N):
        # for m, w in acting_ham(n,V,J,L):
        out_states, out_weights = acting_ham(n,V,J,L)
        # print(np.shape(out_weights), out_weights)
        for ii, m in enumerate(out_states):
            ind2 = basis_N.index(m)
            HN[ind2, ind1] += out_weights[ii]
            # print(n,",",m," -> (", ind2,",",ind1,") -> ", out_weights[ii])
                    
    return HN


def basis_set_NK(N,K,L): #not completed yet
    
    basis_NK = []
    for n in basis_set_N(N,L):
        # print( "-", n)
        bit_n = np.array([int(i) for i in np.binary_repr(n, width=L)])
        bit_m = [np.roll(bit_n, -i) for i in range(L)]
        # m = np.min( [int("".join(str(i) for i in bit),2) for bit in bit_m] )
        m = [int("".join(str(i) for i in bit),2) for bit in bit_m]
        # print( "  -", m)
        # bin_state = np.array([int(i) for i in np.binary_repr(n, width=L)])
        # if sum(bin_state) == N:
        #     basis_NK.append(n)
        
    return 


def particle_count(index, L, parameters, K = 2):
    N = L//2
    # energies, vectors = np.linalg.eigh(build_hamiltonian(N,L,parameters)) 
    energies, vectors = sp.linalg.eigh(build_hamiltonian(N,L,parameters), subset_by_index=[0,K-1]) 
    
    particle_stat = np.zeros((K,L))
    for w, n in enumerate(basis_set_N(N,L)):
        for k in range(K):
            particle_stat[k,:] += (np.conjugate(vectors[w,k])*vectors[w,k]) * np.array([int(i) for i in np.binary_repr(n, width=L)])
    
    return particle_stat, energies

bb = np.array(basis_set_N(3,6))
# bb = basis_set_N(8,16)
print(len(bb))
# aa = [acting_ham(nn,0.314,1,4) for nn in range(2**4)]
# aa = [acting_ham(nn,0.314,1,4) for nn in bb]
# print(aa)

outPut = build_hamiltonian(2,4,(-.314,1))
# print(build_hamiltonian(2,4,(0.314,1)))
print(np.sort( np.linalg.eigvals(outPut)))
particle_count(1,4,(-.0314,1.))

20
[-1.92303997 -1.92303997  0.157       0.157       2.08003997  2.08003997]


(array([[0.73690878, 0.26309122, 0.73690878, 0.26309122],
        [0.26309122, 0.73690878, 0.26309122, 0.73690878]]),
 array([-1.99216541, -1.99216541]))

In [39]:
aa = 1
# np.left_shift(aa,1)
aa>>1

0

# *2D case:* sector construction # 

In [71]:
def bit_flip_2D(input, indxs1:tuple, indxs2:tuple):
    # y1, x1 = indxs1
    # y2, x2 = indxs2
    temp_state = np.array(input[:])
    temp_state[indxs1] = 1 - temp_state[indxs1]
    temp_state[indxs2] = 1 - temp_state[indxs2]
    return temp_state


def basis_set_2D(L):
    N = L//2
    basis_N = []
    for n in range(2**L):
        bin_state = np.array([int(i) for i in np.binary_repr(n, width=L)[::-1]])
        if sum(bin_state) == N:
            basis_N.append(n)
        
    return basis_N


def acting_ham_2D(n:int, dims:tuple, *physical):
    Lx, Ly = dims
    V, J = physical[:2]
    # state = [] # weight = []
    state_weight = []
    # state_weight = {}
    
    bin_state = np.array([int(i) for i in np.binary_repr(n, width = Lx*Ly)[::-1]]).reshape(Ly,Lx)
    
    diag = 0
    for ly in range(Ly):
        rolled = np.roll(bin_state[ly,:], -1)
        diag += - 0.5 * V * (rolled @ bin_state[ly,:])
        diag += - 0.5 * V * (bin_state[ly,:] @ bin_state[np.mod(ly+1,Ly),:])
    
    if Ly == 2:   
        diag += + 0.5 * V * (bin_state[0,:] @ bin_state[1,:])
    
    # if n not in np.transpose(state_weight)[0]:              
    #             state_weight.append( [n, diag] )        
    state_weight.append([n, diag])
    # state_weight[n] = diag
    

    for yy in range(Ly):
        for xx in range(Lx):
            indx1 = (yy,xx)
            indx2 = (yy,np.mod(xx+1,Lx))
            print("   -loop")
            if bin_state[indx1] != bin_state[indx2]:
                # print( bin_state )
                new_state = bit_flip_2D(bin_state, indx1, indx2)
                # print( new_state )
                m = int("".join(str(i) for i in new_state.reshape(-1)[::-1] ), 2)
                st,en = min(indx1[1]+Lx*indx1[0], indx2[1]+Lx*indx2[0])+1, max(indx1[1]+Lx*indx1[0], indx2[1]+Lx*indx2[0]) 
                # print("     -", bin_state.reshape(-1) )
                # print("     -", bin_state.reshape(-1)[st:en]," > (", st, ":", en,")" )
                sign_factor = sum( bin_state.reshape(-1)[st : en])
                if m not in np.transpose(state_weight)[0]:              
                    state_weight.append( [m ,-1.0*J*(-1)**sign_factor] )
                # if m not in state_weight.keys(): 
                #     state_weight[m] = -1.0*J*(-1)**sign_factor
            
            
            # if bin_state[indx1] != bin_state[yy,np.mod(xx-1,Lx)]:
            #     indx2 = (yy,np.mod(xx-1,Lx))
            #     new_state = bit_flip_2D(bin_state, indx1, indx2)
            #     m = int("".join(str(i) for i in new_state.reshape(-1)[::-1] ), 2)
            #     sign_factor = int( 0 )
            #     if m not in np.transpose(state_weight)[0]:               
            #         state_weight.append( [m ,-1.0*J*(-1)**sign_factor] )
            #     # if m not in state_weight.keys(): 
            #     #     state_weight[m] = -1.0*J*(-1)**sign_factor
            
            
            indx2 = (np.mod(yy+1,Ly),xx)
            if bin_state[indx1] != bin_state[indx2]:
                print("------|", indx1, ",",indx2 )
                print( bin_state )
                new_state = bit_flip_2D(bin_state, indx1, indx2)
                print( new_state )
                m = int("".join(str(i) for i in new_state.reshape(-1)[::-1] ), 2)
                
                # st,en = min(indx1[1]+Ly*indx1[0], indx2[1]+Ly*indx2[0])+1, max(indx1[1]+Ly*indx1[0], indx2[1]+Ly*indx2[0]) 
                st,en = min(indx1[1]+Lx*indx1[0], indx2[1]+Lx*indx2[0])+1, max(indx1[1]+Lx*indx1[0], indx2[1]+Lx*indx2[0]) 
                print("     -|", bin_state.reshape(-1) )
                print("     -|", sum( bin_state.reshape(-1)[st : en])," > (", st, ":",en,")" )
                sign_factor = sum( bin_state.reshape(-1)[st : en])
                if m not in np.transpose(state_weight)[0]:             
                    state_weight.append( [m ,-1.0*J*(-1)**sign_factor] )
                # if m not in state_weight.keys(): 
                #     state_weight[m] = -1.0*J*(-1)**sign_factor

            
            # if bin_state[indx1] != bin_state[np.mod(yy-1,Ly),xx]:
            #     indx2 = (np.mod(yy-1,Ly),xx)
            #     new_state = bit_flip_2D(bin_state, indx1, indx2)
            #     m = int("".join(str(i) for i in new_state.reshape(-1)[::-1] ), 2)
            #     sign_factor = int( 0 )
            #     if m not in np.transpose(state_weight)[0]: 
            #         state_weight.append( [m ,-1.0*J*(-1)**sign_factor] )
            #     # if m not in state_weight.keys(): 
            #     #     state_weight[m] = -1.0*J*(-1)**sign_factor

                
    print("-end")
        
    return state_weight
    # return state, weight



L=6
# basis = basis_set_N(L//2,L)
# print(basis)

Lx, Ly = 3, 2

# lissbin = np.array([int(i) for i in np.binary_repr(16, width = Lx*Ly)[::-1]]).reshape(Ly,Lx)
# print(lissbin)
# bit_flip_2D(lissbin,(1,1),(2,1))
# aa = [ float(acting_ham_2D(nn, (Lx, Ly) ,1,0,8)[0][1]) for nn in range(2**(Lx*Ly))]
# print(aa[::-1])

xx = acting_ham_2D(14, (Lx, Ly) ,1,1,8)
# acting_ham_2D(16, (Lx, Ly) ,1,1,8)
print(xx)
print(np.transpose(xx)[0])

   -loop
------| (0, 0) , (np.int64(1), 0)
[[0 1 1]
 [1 0 0]]
[[1 1 1]
 [0 0 0]]
     -| [0 1 1 1 0 0]
     -| 2  > ( 1 : 3 )
   -loop
------| (0, 1) , (np.int64(1), 1)
[[0 1 1]
 [1 0 0]]
[[0 0 1]
 [1 1 0]]
     -| [0 1 1 1 0 0]
     -| 2  > ( 2 : 4 )
   -loop
------| (0, 2) , (np.int64(1), 2)
[[0 1 1]
 [1 0 0]]
[[0 1 0]
 [1 0 1]]
     -| [0 1 1 1 0 0]
     -| 1  > ( 3 : 5 )
   -loop
------| (1, 0) , (np.int64(0), 0)
[[0 1 1]
 [1 0 0]]
[[1 1 1]
 [0 0 0]]
     -| [0 1 1 1 0 0]
     -| 2  > ( 1 : 3 )
   -loop
------| (1, 1) , (np.int64(0), 1)
[[0 1 1]
 [1 0 0]]
[[0 0 1]
 [1 1 0]]
     -| [0 1 1 1 0 0]
     -| 2  > ( 2 : 4 )
   -loop
------| (1, 2) , (np.int64(0), 2)
[[0 1 1]
 [1 0 0]]
[[0 1 0]
 [1 0 1]]
     -| [0 1 1 1 0 0]
     -| 1  > ( 3 : 5 )
-end
[[14, np.float64(-0.5)], [13, -1.0], [7, np.float64(-1.0)], [28, np.float64(-1.0)], [11, np.float64(1.0)], [42, np.float64(1.0)], [22, -1.0], [38, np.float64(-1.0)]]
[14. 13.  7. 28. 11. 42. 22. 38.]


In [86]:
ss =[-4.5, -3.0, -3.0, -2.0, -3.0, -2.0, -2.0, -1.5, -3.0, -2.0, -1.5, -1.0, -1.5, -1.0, -0.5, -0.5, -3.0, -1.5, -2.0, -1.0, -1.5, -0.5, -1.0, -0.5, -2.0, -1.0, -1.0, -0.5, -0.5, 0.0, 0.0, 0.0, -3.0, -1.5, -1.5, -0.5, -2.0, -1.0, -1.0, -0.5, -2.0, -1.0, -0.5, 0.0, -1.0, -0.5, 0.0, 0.0, -2.0, -0.5, -1.0, 0.0, -1.0, 0.0, -0.5, 0.0, -1.5, -0.5, -0.5, 0.0, -0.5, 0.0, 0.0, 0.0]

uq, uq_in = np.unique(ss, return_index=True)

print(uq_in)
print(uq)

indxx = 2,3

srt = np.arange(15).reshape(3,5)#"abcdefghi"
srt[indxx]
srt.reshape(-1, order='A')

new_dict={}
print(new_dict)
new_dict[1]=0.0
new_dict[2]=-1.0
print(new_dict.keys())
print(new_dict.values())

[ 0  1  3  7 11 14 29]
[-4.5 -3.  -2.  -1.5 -1.  -0.5  0. ]
{}
dict_keys([1, 2])
dict_values([0.0, -1.0])


In [33]:
ind11 = (0,3)
ind22 = (0,5)
ind11 + ind22

(0, 3, 0, 5)